In [1]:
import numpy as np
import os
from scipy.misc import imread
import cPickle as pkl
import scipy.signal
import numpy.fft as fft
import sys
import optparse
from PIL import Image
import re
import itertools
from scipy import ndimage

import time
import datetime

import pandas as pd

from bokeh.io import gridplot, output_file, show
from bokeh.plotting import figure
import csv

import pymworks 
import pandas as pd
import operator
import codecs

import scipy.io
import copy# Abstract struct class       

class Struct:
    def __init__ (self, *argv, **argd):
        if len(argd):
            # Update by dictionary
            self.__dict__.update (argd)
        else:
            # Update by position
            attrs = filter (lambda x: x[0:2] != "__", dir(self))
            for n in range(len(argv)):
                setattr(self, attrs[n], argv[n])


class cycstruct(Struct):
    states = []
    idxs = 0
    vals = 0
    ordernum = 0
    triggers = 0


In [2]:
from parse_mw_events import get_bar_events

In [3]:
prepend = '/'; #'/Users/julianarhee'
source_dir = os.path.join(prepend,'nas/volume1/2photon/RESDATA/20170811_CE052/retinotopy3') #options.source_dir #'/nas/volume1/2photon/RESDATA/TEFO/20160118_AG33/fov1_gratings1'
stimtype = 'bar' #options.stimtype #'grating'
mask = False #options.mask # False
long_trials = False #options.long_trials #True
no_ard = False #options.no_ard

# Look in child dir (of source_dir) to find mw_data paths:
mw_files = os.listdir(os.path.join(source_dir, 'mw_data'))
mw_files = [m for m in mw_files if m.endswith('.mwk')]


In [4]:
mwfile = mw_files[0]

fn_base = mwfile[:-4]
mw_data_dir = os.path.join(source_dir, 'mw_data')
mw_fn = fn_base+'.mwk'
dfn = os.path.join(mw_data_dir, mw_fn)
dfns = [dfn]

print "MW file: ", dfns

MW file:  ['/nas/volume1/2photon/RESDATA/20170811_CE052/retinotopy3/mw_data/20170811_CE052_run3.mwk']


In [5]:
# Get MW events:
if stimtype=='bar':
    pixelevents, stimevents, trigger_times, session_info = get_bar_events(dfns[0])
else:
    pixelevents, stimevents, trialevents, trigger_times, session_info = get_stimulus_events(dfns, stimtype=stimtype)

    

****************************************************************
Parsing file
/nas/volume1/2photon/RESDATA/20170811_CE052/retinotopy3/mw_data/20170811_CE052_run3.mwk... 
Found 1 start events in session.
Bounds:  [[457767409969, 458175153294]]
bound ID: 0 407.743325 sec
****************************************************************
................................................................
SECTION 0
................................................................
Found > 1 name for frame-trigger:
Choose:  ['frame_trigger_timeout', 'frame_trigger_on', 'trigger_received', 'trigger_active', 'frame_trigger']
Hint: RSVP could be FrameTrigger, otherwise frame_trigger.
Type var name to use: frame_trigger
first on event:  Event[code=39, name=frame_trigger, time=457780845705, value=0]
first off event:  Event[code=39, name=frame_trigger, time=457855277621, value=1]
Duration of first run: 74.4319 sec.
[Event[code=39, name=frame_trigger, time=457767410045, value=1], Event[code=39, name=fram

In [6]:
# Make sure only 1 "chunk" of pixelevents are found for the session:
if len(pixelevents) > 1:
    pixelevents = [item for sublist in pixelevents for item in sublist]
    pixelevents = list(set(pixelevents))
    pixelevents.sort(key=operator.itemgetter(1))
else:
    pixelevents = pixelevents[0]
print "Found %i pixel clock events." % len(pixelevents)

if len(stimevents) > 1:
    stimevents = [item for sublist in stimevents for item in sublist]
    stimevents = list(set(stimevents))
    stimevents.sort(key=operator.itemgetter(1))
else:
    stimevents = stimevents[0]
print "Found %i stimulus on events." % len(stimevents)

if len(trigger_times) > 1:
    trigger_times = [item for sublist in trigger_times for item in sublist]
    trigger_times = list(set(trigger_times))
    trigger_times.sort(key=operator.itemgetter(1))
else:
    trigger_times = trigger_times[0]
print "Found %i runs (i.e., trigger boundary events)." % len(trigger_times)

if len(session_info) > 1:
    session_info = [item for sublist in session_info for item in sublist]
    session_info = list(set(trigger_times))
    session_info.sort(key=operator.itemgetter(1))
else:
    session_info = session_info[0]
print "N sessions in file: ." % len(session_info)
print session_info



Found 17760 pixel clock events.


In [7]:
print len(stimevents)

4


In [8]:
# Check that all possible pixel vals are used (otherwise, pix-clock may be missing input):
# pixelevents[0].value[-1]['bit_code']
n_codes = set([i.value[-1]['bit_code'] for i in pixelevents])
if len(n_codes)<16:
    print "Check pixel clock -- missing bit values..."


In [9]:
# Creat trial-dicts for each trial in run, in format for NDA database:
if stimtype=='bar':
    nexpected_pixelevents = int(round((1/session_info['target_freq']) * session_info['ncycles'] * 60 * len(trigger_times)))
    print "Expected %i pixel events, missing %i pevs." % (nexpected_pixelevents, nexpected_pixelevents-len(pixelevents))
    # on FLASH protocols, first real iamge event is 41
    print "Found %i conditions, corresponding to %i TIFFs." % (len(stimevents), len(trigger_times))

    stimnames = ['left', 'right', 'top', 'bottom']

    # GET TRIAL INFO FOR DB:
    trial_list = [(stimevents[k].ordernum, k) for k in stimevents.keys()]
    trial_list.sort(key=lambda x: x[0]) 
    trial = dict((i+1, dict()) for i in range(len(stimevents)))

    for trialidx,mvtrial in enumerate(trial_list):
        mvname = mvtrial[1]
        trialnum = trialidx + 1
        trial[trialnum]['start_time_ms'] = round(stimevents[mvname].states[0][0]/1E3)
        trial[trialnum]['end_time_ms'] = round(stimevents[mvname].states[-1][0]/1E3)
        stimname = [i for i in stimnames if i in mvname][0]
        stimsize = session_info['target_freq']
        trial[trialnum]['stimuli'] = {'stimulus': stimname, 'position': stimevents[mvname].states[0][1], 'scale': stimsize}
        trial[trialnum]['stim_on_times'] = round(stimevents[mvname].states[0][0]/1E3)
        trial[trialnum]['stim_off_times'] = round(stimevents[mvname].states[-1][0]/1E3)
# else:
#     ievs = ievs[0]
#     print "Found %i image trials." % len(stimevents)

#     print "Found %i stimulus update events across trials." % len(trialevents)
#     print "Expecting %i TIFFs." % len(trigger_times)

#     # GET TRIAL INFO FOR DB:
#     trial = dict((i+1, dict()) for i in range(len(stimevents)))
#     stimevents = sorted(stimevents, key=get_timekey)
#     trialevents = sorted(trialevents, key=get_timekey)
#     run_start_time = trialevents[0].time
#     for trialidx,stimevent in enumerate(sorted(stimevents, key=get_timekey)):
#         trialnum = trialidx + 1
#         blankidx = trialidx*2 + 1
#         trial[trialnum]['start_time_ms'] = round(stimevent.time/1E3)
#         trial[trialnum]['end_time_ms'] = round((trialevents[blankidx].time + session_info['ITI'])/1E3)
#         if stimtype=='grating':
#             ori = stimevent.value[1]['rotation']
#             sf = round(stimevent.value[1]['frequency'], 2)
#             stimname = 'grating-ori-%i-sf-%f' % (ori, sf)
#             stimpos = [stimevent.value[1]['xoffset'], stimevent.value[1]['yoffset']]
# #         else:
# #             stimname = ''
# #             stimpos = ''
#         stimsize = stimevent.value[1]['height']
#         trial[trialnum]['stimuli'] = {'stimulus': stimname, 'position': stimpos, 'scale': stimsize}
#         trial[trialnum]['stim_on_times'] = round((stimevent.time - run_start_time)/1E3)
#         trial[trialnum]['stim_off_times'] = round((trialevents[blankidx].time - run_start_time)/1E3)


Expected 17778 pixel events, missing 18 pevs.
Found 4 conditions, corresponding to 4 TIFFs.


In [10]:
print "Format trial 1: ", trial[1]
# save trial info as pkl for easyloading: 
trialinfo_fn = 'trial_info.pkl'
with open(os.path.join(source_dir, 'mw_data', trialinfo_fn), 'wb') as f:
    pkl.dump(trial, f, protocol=pkl.HIGHEST_PROTOCOL)
    f.close()


Format trial 1:  {'stim_off_times': 457854860.0, 'stimuli': {'stimulus': 'bottom', 'position': (0.0, -50.1907958984375), 'scale': 0.27}, 'stim_on_times': 457780875.0, 'start_time_ms': 457780875.0, 'end_time_ms': 457854860.0}


In [11]:
# May need to fix this:
print "Each chunk duration is: ", [(t[1] - t[0])/1E6 for t in trigger_times]


Each chunk duration is:  [74.431916, 74.448072, 74.456103, 74.447903]


In [24]:
# Create "pydict" to store all MW stimulus/trial info in matlab-accessible format for GUI:
if stimtype=='bar':
    pydict = dict()
    print "Offset between first MW stimulus-display-update event and first SI frame-trigger:"
    for ridx,run in enumerate(stimevents.keys()):
        pydict[run] ={'time': [i[0] for i in stimevents[run].states],\
                    'pos': stimevents[run].vals,\
                    'idxs': stimevents[run].idxs,\
                    'ordernum': stimevents[run].ordernum,\
                    'MWdur': (stimevents[run].states[-1][0] - stimevents[run].states[0][0]) / 1E6,\
                    'offset': stimevents[run].states[0][0] - stimevents[run].triggers[0],\
                    'MWtriggertimes': stimevents[run].triggers}
        print "run %i: %s ms" % (ridx+1, str(pydict[run]['offset']/1E3))
elif stimtype == 'image':
    # do stuff
    pass
elif stimtype == 'grating':
    # do other stuff
    pass

Offset between first MW stimulus-display-update event and first SI frame-trigger:
run 1: 27.609 ms
run 2: 18.0 ms
run 3: 32.62 ms
run 4: 29.54 ms


In [25]:
pydict['bottom']['time']

[457780875245,
 457780891912,
 457780908579,
 457780925246,
 457780941914,
 457780958581,
 457780975248,
 457780991915,
 457781008582,
 457781025249,
 457781041916,
 457781058583,
 457781075250,
 457781091917,
 457781108584,
 457781125251,
 457781141918,
 457781158585,
 457781175252,
 457781191919,
 457781208586,
 457781225253,
 457781241920,
 457781258587,
 457781275254,
 457781291921,
 457781308588,
 457781325255,
 457781341922,
 457781358589,
 457781375256,
 457781391924,
 457781408591,
 457781425258,
 457781441925,
 457781458592,
 457781475259,
 457781491926,
 457781508593,
 457781525260,
 457781541927,
 457781558594,
 457781575261,
 457781591928,
 457781608595,
 457781625262,
 457781641929,
 457781658596,
 457781675263,
 457781691930,
 457781708597,
 457781725264,
 457781741931,
 457781758598,
 457781775265,
 457781791932,
 457781808599,
 457781825266,
 457781841934,
 457781858601,
 457781875268,
 457781891935,
 457781908602,
 457781925269,
 457781941936,
 457781958603,
 457781975

In [26]:
stimevents['bottom'].states

[[457780875245, (0.0, -50.1907958984375)],
 [457780891912, (0.0, -49.79441833496094)],
 [457780908579, (0.0, -49.36809158325195)],
 [457780925246, (0.0, -48.909698486328125)],
 [457780941914, (0.0, -48.45108413696289)],
 [457780958581, (0.0, -47.98051071166992)],
 [457780975248, (0.0, -47.51424789428711)],
 [457780991915, (0.0, -47.057498931884766)],
 [457781008582, (0.0, -46.58970642089844)],
 [457781025249, (0.0, -46.131813049316406)],
 [457781041916, (0.0, -45.66950225830078)],
 [457781058583, (0.0, -45.19692611694336)],
 [457781075250, (0.0, -44.7397575378418)],
 [457781091917, (0.0, -44.27883529663086)],
 [457781108584, (0.0, -43.8079833984375)],
 [457781125251, (0.0, -43.34886932373047)],
 [457781141918, (0.0, -42.88224411010742)],
 [457781158585, (0.0, -42.417537689208984)],
 [457781175252, (0.0, -41.95174789428711)],
 [457781191919, (0.0, -41.47074890136719)],
 [457781208586, (0.0, -41.00896072387695)],
 [457781225253, (0.0, -40.55986022949219)],
 [457781241920, (0.0, -40.08266

In [27]:
# arduino stuff.. skip for now.

In [28]:
session_info

{'barwidth': 2.5, 'ncycles': 20, 'target_freq': 0.27}

In [29]:
    pydict['stimtype'] = stimtype

    #pydict['mw_times_by_file'] = mw_times_by_file
    #pydict['mw_file_durs'] = mw_file_durs
    #pydict['mw_frame_trigger_times'] = frame_trigger_times
    #pydict['offsets_by_file'] = offsets_by_file
    #pydict['mw_codes_by_file'] = mw_codes_by_file
    pydict['mw_dfn'] = dfn
    pydict['source_dir'] = source_dir
    pydict['fn_base'] = fn_base
    pydict['stimtype'] = stimtype
    if stimtype=='bar':
        pydict['condtypes'] = ['left', 'right', 'top', 'bottom']
        pydict['runs'] = stimevents.keys()
        pydict['info'] = session_info
#     elif stimtype=='image':
#         pydict['condtypes'] = sorted(image_ids)
#         pydict['runs'] = stimevents.keys()
#     elif stimtype=='grating':
#         pydict['condtypes'] = sorted(image_ids)
#         pydict['runs'] = stimevents.keys()

    tif_fn = fn_base+'.mat'
    # scipy.io.savemat(os.path.join(source_dir, condition, tif_fn), mdict=pydict)
    scipy.io.savemat(os.path.join(source_dir, 'mw_data', tif_fn), mdict=pydict)
    print os.path.join(source_dir, 'mw_data', tif_fn)


/nas/volume1/2photon/RESDATA/20170811_CE052/retinotopy3/mw_data/20170811_CE052_run3.mat
